# Toggle between CPU and GPU(CUDA)

In [1]:
USE_CUDA = True

# Install Python libraries

In [2]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install tqdm torchtext
!pip install gputil
!pip install psutil
!pip install humanize
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Download additional libraries and data

In [3]:
!wget -O attention.py https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/attention.py && rm attention.pyc
!wget -O model.py https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/model.py && rm model.pyc
!wget -O rouge.py https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/rouge.py && rm rouge.pyc

--2018-03-25 14:31:07--  https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/attention.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6831 (6.7K) [text/plain]
Saving to: ‘attention.py’

attention.py        100%[===================>]   6.67K  --.-KB/s    in 0s      

2018-03-25 14:31:07 (94.5 MB/s) - ‘attention.py’ saved [6831/6831]

rm: cannot remove 'attention.pyc': No such file or directory
--2018-03-25 14:31:07--  https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|

In [4]:
!wget -O batch_100_download.sh https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/batch_100/download.sh
!wget -O batch_1000_download.sh https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/batch_1000/download.sh
!wget -O cnn_test_stories.txt https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/cnn_test_stories.txt

--2018-03-25 14:31:11--  https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/batch_100/download.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 234 [text/plain]
Saving to: ‘batch_100_download.sh’

batch_100_download. 100%[===================>]     234  --.-KB/s    in 0s      

2018-03-25 14:31:11 (62.8 MB/s) - ‘batch_100_download.sh’ saved [234/234]

--2018-03-25 14:31:11--  https://raw.githubusercontent.com/mattheuslee/Deep-Reinforced-Abstractive-Summarization/master/batch_1000/download.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP reques

Run download script to get batch 100 or 1000

In [5]:
!bash batch_100_download.sh

mkdir: cannot create directory ‘batch_100’: File exists


In [0]:
!bash batch_1000_download.sh

# Run to check GPU stats and usage

In [2]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
process = psutil.Process(os.getpid())
print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
!nvidia-smi

ln: cannot remove ‘/usr/bin/nvidia-smi’: Permission denied


AttributeError: 'Process' object has no attribute 'memory_info'

# Main Code

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import nltk
from torchtext.data import Field,BucketIterator, TabularDataset
from tqdm import tqdm, trange, tnrange, tqdm_notebook
import numpy as np
from __future__ import print_function

from model import Encoder, Decoder

In [3]:
TEXT = Field(tokenize=nltk.word_tokenize,use_vocab=True,lower=True, include_lengths=True, batch_first=True)

In [4]:
MINI_BATCH_SIZE = 1

**Change path if using different data file**

In [18]:
!ls

attention.py   batch_1000	       cnn_test_stories.txt  model.pyc
attention.pyc  batch_1000_download.sh  datalab		     nltk_data
batch_100      batch_100_download.sh   model.py		     rouge.py


In [5]:
BATCH_SIZE  = 100
NUM_BATCHES = 800

train_data_batches = []
for batch_idx in trange(NUM_BATCHES, desc = "Loading batches", unit = "batch"):
    train_data_batches.append(TabularDataset(path = "batch_{}/cnn_stories_80000_{}.txt".format(BATCH_SIZE, batch_idx),
                                             format = "tsv",
                                             fields = [("input", TEXT), ("target", TEXT)]))

test_data  = TabularDataset(path="cnn_test_stories.txt",
                            format='tsv',
                            fields=[('input',TEXT),('target',TEXT)])

TEXT.build_vocab(test_data, *train_data_batches, min_freq=2)
tqdm.write("Vocabulary size: {}".format(len(TEXT.vocab)))

batch_train_loaders = []
for batch_idx in trange(NUM_BATCHES, desc = "Batch training dataloaders", unit = "batch"):
    batch_train_loaders.append(BucketIterator(train_data_batches[batch_idx], 
                               batch_size = MINI_BATCH_SIZE,
                               device = -1,
                               sort_key = lambda x: len(x.input),
                               sort_within_batch = True,
                               repeat = False,
                               shuffle = True))

test_loader  = BucketIterator(test_data,
                              batch_size = 1, 
                              device = -1,
                              sort_key = lambda x: len(x.input),
                              sort_within_batch = True,
                              repeat = False,
                              shuffle = True)

# May be slightly less due to skipping empty stories
num_training_examples = np.sum([len(t) for t in train_data_batches])
tqdm.write("Number of training examples: {}".format(num_training_examples))
tqdm.write("Number of testing stories: {}".format(len(test_data)))

HIDDEN = 200
EMBED = 100
VOCAB_SIZE = len(TEXT.vocab)
LR = 0.001

Batch training dataloaders: 100%|██████████| 800/800 [00:00<00:00, 42399.90batch/s]

Vocabulary size: 188274
Number of training examples: 79903
Number of testing stories: 10


In [6]:
encoder = Encoder(VOCAB_SIZE,EMBED,HIDDEN,bidirec=True)
decoder = Decoder(VOCAB_SIZE,EMBED,HIDDEN*2)

### Load/save model

In [22]:
encoder.load_state_dict(torch.load("./encoder.model"))
decoder.load_state_dict(torch.load("./decoder.model"))

IOError: ignored

In [0]:
torch.save(encoder.state_dict(), "./encoder.model")
torch.save(decoder.state_dict(), "./decoder.model")

In [7]:
if USE_CUDA:
    tqdm.write("Using CUDA")
    if torch.cuda.device_count() > 1:
        print("Using %d devices" % (torch.cuda.device_count()))
        encoder = nn.DataParallel(encoder)
        decoder = nn.DataParallel(decoder)
    encoder = encoder.cuda()
    decoder = decoder.cuda()
decoder.embedding = encoder.embedding

Using CUDA


In [8]:
loss_function = nn.CrossEntropyLoss(ignore_index=TEXT.vocab.stoi['<pad>'])
enc_optim = optim.Adam(encoder.parameters(),lr=LR)
dec_optim = optim.Adam(decoder.parameters(),lr=LR)

### Run training

In [10]:
NUM_EPOCHS = 1
for epoch_idx in trange(NUM_EPOCHS, desc = "Epochs", unit = "epoch"):
    total_loss, total_squared_loss, num_batches = 0.0, 0.0, 0
    for batch_train_loader in tqdm(batch_train_loaders, desc = "Batches", unit = "batch"):
        for minibatch in tqdm(batch_train_loader, desc = "Minibatches", unit = "minibatch"):
            inputs,lengths = minibatch.input
            targets,_ = minibatch.target
            decoding_start = Variable(torch.LongTensor([TEXT.vocab.stoi['<s>']]*targets.size(0))).unsqueeze(1)
            if USE_CUDA:
                inputs = inputs.cuda()
                targets = targets.cuda()
                decoding_start = decoding_start.cuda()

            encoder.zero_grad()
            decoder.zero_grad()
            output,hidden = encoder(inputs,lengths.tolist())
            score = decoder(decoding_start,hidden,targets.size(1),output,lengths)

            loss = loss_function(score,targets.view(-1))
            total_loss += loss.data[0]
            total_squared_loss += loss.data[0]**2
            num_batches += 1
            loss.backward()
            enc_optim.step()
            dec_optim.step()
    loss_mean = total_loss / num_batches
    loss_variance = (total_squared_loss - (total_loss**2 / num_batches)) / (num_batches - 1)
    tqdm.write("loss mean: %7.4f, loss variance: %7.4f" % (loss_mean, loss_variance))

Batches:   0%|          | 0/800 [00:00<?, ?batch/s]

Minibatches:   0%|          | 0/100 [00:00<?, ?minibatch/s]





RuntimeError: cuda runtime error (2) : out of memory at /pytorch/torch/lib/THC/generic/THCTensorMath.cu:35

###Run testing

In [0]:
from rouge import ROUGE
from __future__ import print_function
rouge = ROUGE()

print(TEXT.vocab.)

def get_string(summary):
    result = ""
    for idx in summary:
        if idx == 1: # <E>
            break
        elif idx == 0: # <unk>
            continue
        if idx < len(TEXT.vocab.itos):
            result += (TEXT.vocab.itos[idx] + " ")
    return result

encoder.eval()
decoder.eval()
print("Testing stories")
for batch in test_loader:
    inputs, lengths = batch.input
    targets, _ = batch.target
    decoding_start = Variable(torch.LongTensor([TEXT.vocab.stoi['<s>']]*targets.size(0))).unsqueeze(1)
    if USE_CUDA:
        inputs = inputs.cuda()
        targets = targets.cuda()
        decoding_start = decoding_start.cuda()
    
    output,hidden = encoder(inputs,lengths.tolist())
    score = decoder(decoding_start,hidden,targets.size(1),output,lengths)
    
    reference_summary = targets.data.cpu().numpy()[0]
    generated_summary = [np.argmax(word) for word in score.data.cpu().numpy()[0]]
    
    reference = get_string(reference_summary)
    generated = get_string(generated_summary)
    
    rouge_score = rouge.score(reference, generated)
    
    print("\nReference summary:\n{}".format(reference))
    print("\nGenerated summary:\n{}".format(generated))
    print("\nROUGE score: {}\n".format(rouge_score))

print("Selection of training stories")
NUM_SHOW_TRAINING_STORIES = 10
for i, batch in enumerate(train_loader):
    if i == NUM_SHOW_TRAINING_STORIES:
        break
    inputs, lengths = batch.input
    targets, _ = batch.target
    decoding_start = Variable(torch.LongTensor([TEXT.vocab.stoi['<s>']]*targets.size(0))).unsqueeze(1)
    if USE_CUDA:
        inputs = inputs.cuda()
        targets = targets.cuda()
        decoding_start = decoding_start.cuda()
    
    output,hidden = encoder(inputs,lengths.tolist())
    score = decoder(decoding_start,hidden,targets.size(1),output,lengths)
    
    reference_summary = targets.data.cpu().numpy()[0]
    generated_summary = [np.argmax(word) for word in score.data.cpu().numpy()[0]]
    
    reference = get_string(reference_summary)
    generated = get_string(generated_summary)
    
    rouge_score = rouge.score(reference, generated)
    
    print("\nReference summary:\n{}".format(reference))
    print("\nGenerated summary:\n{}".format(generated))
    print("\nROUGE score: {}\n".format(rouge_score))
    

NameError: ignored